In [4]:
#importando pacotes para conexão com o dynamo
import boto3
from boto3.dynamodb.conditions import Attr
from datetime import datetime
import requests

In [5]:
#conectando ao dynamo
dynamodb = boto3.resource('dynamodb', 
                          region_name='us-east-1', 
                          aws_access_key_id='#######', 
                          aws_secret_access_key='######')

In [6]:
#fazendo a query para pegar os cpfs consultados no boa vista
table = dynamodb.Table('applicant-bureau-api-data-prod')
f = Attr('bureau_client_solicitation_ids').contains('serasa')
response = table.scan(FilterExpression=f)

In [7]:
#percorrendo os resultados
while 'LastEvaluatedKey' in response:
    for i in response['Items']:
        dataAnaliseCPF = datetime.strptime(i['dated'], '%Y-%m-%d %H:%M:%S').date()
        dataLimite = datetime.strptime('2021-11-09 00:00:00', '%Y-%m-%d %H:%M:%S').date()
        if(dataAnaliseCPF >= dataLimite):
            #procurando se existe consultas anteriores
            if 'registros' in i['data']:
                print('O CPF ' + i['document'] + ' contém consultas anteriores')
                print(dataAnaliseCPF)
            #procurando se existe pendencias financeiras
            if 'pendencia_pagamento' in i['data']:
                print('O CPF ' + i['document'] + ' contém pendências financeiras')
                print(dataAnaliseCPF)
            #procurando se existe cheques sem fundo
            if 'cheques' in i['data']:
                print('O CPF ' + i['document'] + ' contém cheques sem fundo')
                print(dataAnaliseCPF)
            print('---')
    if 'LastEvaluatedKey' in response:
        response = table.scan(FilterExpression=f, ExclusiveStartKey=response['LastEvaluatedKey'])

O CPF 72551984904 contém pendências financeiras
2021-11-09
---
O CPF 43248392800 contém consultas anteriores
2021-11-09
---
---
---
---
O CPF 04696293637 contém pendências financeiras
2021-11-09
---
O CPF 38251620600 contém consultas anteriores
2021-11-09
---
O CPF 04882844656 contém consultas anteriores
2021-11-09
---
O CPF 29881232848 contém pendências financeiras
2021-11-09
---
---
O CPF 61824615000 contém consultas anteriores
2021-11-09
---
---
---
O CPF 04800573971 contém consultas anteriores
2021-11-09
---
---
---
O CPF 18373994840 contém pendências financeiras
2021-11-09
---
O CPF 41365445100 contém pendências financeiras
2021-11-09
---
---
---
O CPF 19132218818 contém consultas anteriores
2021-11-09
---
---
---
---
O CPF 05801387986 contém consultas anteriores
2021-11-09
---
O CPF 84053364191 contém consultas anteriores
2021-11-09
---
O CPF 26020680800 contém consultas anteriores
2021-11-09
---
O CPF 82573344991 contém consultas anteriores
2021-11-09
O CPF 82573344991 contém pe

---
---
---
O CPF 52275728953 contém consultas anteriores
2021-11-09
---
O CPF 17022493871 contém consultas anteriores
2021-11-09
---
O CPF 06111062999 contém consultas anteriores
2021-11-09
---
---
---
O CPF 00278458009 contém consultas anteriores
2021-11-09
---
---
---
---
---
---
O CPF 04118009900 contém consultas anteriores
2021-11-09
---
---
O CPF 25808478987 contém consultas anteriores
2021-11-09
---
O CPF 04920199163 contém pendências financeiras
2021-11-09
---
---
O CPF 65929578320 contém consultas anteriores
2021-11-09
---
O CPF 38646137953 contém consultas anteriores
2021-11-09
O CPF 38646137953 contém pendências financeiras
2021-11-09
---
---
---
O CPF 01780064918 contém consultas anteriores
2021-11-09
---
O CPF 14742580770 contém pendências financeiras
2021-11-09
---
---
O CPF 83276513749 contém consultas anteriores
2021-11-09
---
---
O CPF 08981718733 contém pendências financeiras
2021-11-09
---
O CPF 09182030901 contém consultas anteriores
2021-11-09
O CPF 09182030901 con

In [8]:
#BOA VISTA - conectando a api, capturando o retorno e mostrando o resultado
#url_api_boa_vista = "https://www.scpc.inf.br/cgi-bin/spcaxml"
#header = { 'Accept': 'application/text' }
#login = '18641'
#senha = 'FC#2005'
#produto = '309'

#cpfs = ['72739495072']
#for cpf in cpfs:
#    body = "<SPCA-XML xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance' xsi:noNamespaceSchemaLocation='https://www.scpc.inf.br/spcn/spcaxmldef.xsd'>  	 	<VERSAO>20190710</VERSAO>      	<SOLICITACAO>         		<S-REGIONAL>0</S-REGIONAL>         		<S-CODIGO>"+login+"</S-CODIGO>         		<S-SENHA>"+senha+"</S-SENHA>         		<S-CONSULTA>"+produto+"</S-CONSULTA>         		<S-SOLICITANTE>FULANO DE TAL</S-SOLICITANTE>         		<S-CPF>"+cpf+"</S-CPF>  		<S-BANCO>1</S-BANCO> 		<S-AGENCIA>1</S-AGENCIA> 		<S-CONTA-CORRENTE>1</S-CONTA-CORRENTE> 		<S-CHEQUE>1</S-CHEQUE> 		<S-CHEQUE-QTDE>1</S-CHEQUE-QTDE> 		<S-CHEQUE-DIGITO>9</S-CHEQUE-DIGITO> 		<S-DDD-1>99</S-DDD-1>         		<S-TELEFONE-1>999999999</S-TELEFONE-1>         		<S-CEP>9999999</S-CEP>         		<S-ACERTA-PRODUTO>             			<S-ACERTA-SCORE>S</S-ACERTA-SCORE> 			<S-ACERTA-SCORE-12>N</S-ACERTA-SCORE-12> 			<S-ACERTA-SCORE-CC>N</S-ACERTA-SCORE-CC> 			<S-ACERTA-SCORE-63>S</S-ACERTA-SCORE-63>   			<S-ACERTA-CHEQUE>S</S-ACERTA-CHEQUE>     		</S-ACERTA-PRODUTO>  	 	</SOLICITACAO>  </SPCA-XML>"
#    response_boa_vista = requests.get(url_api_boa_vista, headers=header, data=body)
#    print(response_boa_vista.text)